In [ ]:
%matplotlib inline

import hashlib
import pandas as pd
import matplotlib.pyplot as plt

idx = pd.IndexSlice

In [ ]:
data_path = 'https://raw.githubusercontent.com/vohcolab/PandaViz-Workshop/main/Pandas/Time%20Series/data/covid19data.csv'
df = pd.read_csv(data_path)
df.head()

## Exercise 1) Multi Indexing

Convert the date column to datetime and make an index with [date,Country]. Don't forget to sort the index!

In [ ]:
data = df.copy()

data['date'] = pd.to_datetime(data['date'], infer_datetime_format=True)
data = data.set_index(['date', 'Country'])
data = data.sort_index()

In [ ]:
data.head(5)

In [ ]:
expected_hash = 'e0a979cf8955d1e01188e855301a73abba801de72f4a402a85abb547e65c5d21'
assert hashlib.sha256(str(data.head(5)).encode()).hexdigest() == expected_hash
assert hashlib.sha256(str(data.index.dtype).encode()).hexdigest() == '2958d416d08aa5a472d7b509036cb7eafd542add84527e66a145ea64cb4cdc75'

## Exercise 2) Subsetting

#### Exercise 2.1) What day of the month was it when the daily deaths was highest in Panama?

The answer should be a timestamp

In [ ]:
panama_highest_daily_death_count = data.loc[idx[:,'Panama'],'deaths'].idxmax()[0]
# explanation
# data is already daily
# .idxmax() returns the index with highest deaths
# however our index is double-index remeber? so we have something of the form
# (timestamp,location). that's why we then access the first element with [0]

In [ ]:
panama_highest_daily_death_count

In [ ]:
expected_hash = '6f36f9238aa16925dac6ab834afccaddcc3a7487b0e9c868f5c47416f83d3036'

assert hashlib.sha256(str(panama_highest_daily_death_count).encode()).hexdigest() == expected_hash

#### Exercise 2.2) In October 31st, what were the top 5 countries whose confirmed cases rose the most since the day before?

result must be an Index type

In [ ]:
top5 = data.loc[idx['Oct 31st 2020',:],'confirmed_cases'].nlargest(5).index.get_level_values('Country')

In [ ]:
top5

In [ ]:
expected_hash = '7bc8a29be43a8beddc2b8fe5585c2bb7c89930695f80d5e0ce852b01c9789b4d'

assert hashlib.sha256(str(top5).encode()).hexdigest() == expected_hash

#### Exercise 2.3) best day of the week to go for a walk?

It's September at the moment. You and your friends are visiting Spain and are wondering what day of the week is safest to go take a long walk through the city. So you decide to check throughout the whole of August which day of the week has had the least number of daily new confirmed cases in total.

_hint_ you may want to first add a new column with the weekday's name

In [ ]:
data_v2 = data.copy()
data_v2['weekday'] = data_v2.index.get_level_values('date').day_name()
weekday = data_v2.loc[idx['August 2020'], ['confirmed_cases','weekday']].groupby('weekday').sum().idxmin().values[0]
# create weekday column so that we can group by it
# idxmin() for weekday with least confirmed_cases
# values[0] to get the final weekday string
# PS: try removing "values[0]" to see what you get

In [ ]:
weekday

In [ ]:
expected_hash = '7d8af1de1262f150187d5938bf649d6c35726970685f916d1ec6392a801b9762'

assert hashlib.sha256(str(weekday).encode()).hexdigest() == expected_hash

## Exercise 3

#### Exercise 3.1) Top 3 countries with all-time most confirmed cases

The answer should be a list with the 3 countries presented in descending order according to their all-time confirmed cases

In [ ]:
top3_most_confirmed = data.groupby('Country').sum().nlargest(3, columns='confirmed_cases').index.tolist()

In [ ]:
top3_most_confirmed

In [ ]:
expected_hash = '2639a1aee85cdd11e6f24f3a7e0f9c5ca9a23d8d6907d2f2f30c38a63197b9c2'

assert hashlib.sha256(str(top3_most_confirmed).encode()).hexdigest() == expected_hash

#### Exercise 3.2) What's the country where the percentage change of deaths is smallest from  November 2nd to the 3rd?


The answer should be a string.

In [ ]:
interval = slice('November 2nd 2020', 'November 3rd 2020')
country_smallest_death_variation = data.loc[idx[interval,:], 'deaths'].pct_change().idxmin()[1]

In [ ]:
country_smallest_death_variation

In [ ]:
expected_hash = '71e9eb10755c8232974d186a560f7658a1ccbdd3097877dfa1ad3436af90d46a'

assert hashlib.sha256(str(country_smallest_death_variation).encode()).hexdigest() == expected_hash

## Plotting multiple timeseries

You've reach the end. Congratulations! As a reward I'll show you one more trick you can do with plotting in time series.

We can compare data across countries in a very nice way in Pandas. Let's Compare Portugal's with Italy's.

In [ ]:
plt.figure(figsize=(14,6))

october = 'October 2020'
portugal_data = data.loc[idx[october, 'Portugal'], 'deaths'].pct_change()
italy_data = data.loc[idx[october, 'Italy'], 'deaths'].pct_change()

portugal_data.plot(label='Portugal')
italy_data.plot(label='Italy')
plt.title('Daily deaths percentage change for October 2020', fontsize=20)
plt.legend()

hum the x axis is a bit weird..

We wanted to have just the dates, but because we have a double index the x axis also includes the country alongside each date. Which is redundant because we already know which countries we are talking about. 

Lets fix that!

In [ ]:
plt.figure(figsize=(14,6))

october = 'October 2020'
portugal_data = data.loc[idx[october, 'Portugal'], 'deaths'].pct_change()
italy_data = data.loc[idx[october, 'Italy'], 'deaths'].pct_change()

# fix index
portugal_data_fixed, italy_data_fixed = portugal_data.copy(), italy_data.copy()
portugal_data_fixed.index = portugal_data_fixed.index.droplevel('Country') # simply drop country from index and keep only dates
italy_data_fixed.index = italy_data_fixed.index.droplevel('Country') # simply drop country from index and keep only dates

portugal_data_fixed.plot(label='Portugal')
italy_data_fixed.plot(label='Italy')
plt.title('Daily deaths percentage change for October 2020', fontsize=20)
plt.legend()